In [1]:
# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# # Importing the dataset
# dataset = pd.read_csv('movies_merged.csv', sep='|')
# # Remove NaN
# dataset = dataset.dropna()
# # Lower all words
# dataset = dataset.applymap(str.lower)

def read_csv(name, index_col=None):
    df = pd.read_csv(name, low_memory=False, sep='|', index_col=index_col)
    return df

def process_df(name):
    dataset = read_csv(name)
    dataset.dropna(inplace=True)
    dataset['Review'].replace(to_replace='Fresh', value=1, inplace=True)
    dataset['Review'].replace(to_replace='Rotten', value=0, inplace=True)
    return dataset

dataset = process_df('movies_merged.csv')
dataset.head()

,Text,Review
0,Too few films take on the art of arguing as a ...,1
1,The film leaves a tremendous impact.,1
3,From 1957 and first-time director Sidney Lumet...,1
4,"Mechanically written, but within its own middl...",1
5,A strangely realistic thriller.,1


In [2]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
)
features = vectorizer.fit_transform(
    dataset.iloc[:,0]
)

from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test  = train_test_split(
#         features, 
#         dataset.iloc[:,1],
#         train_size=0.80, 
#         random_state=1234)

MAX_WORDS = 1000
MAX_LEN = 150


def get_X_from_df(df):
    X = df.Text
    return X

def get_Y_from_df(df):
    Y = df.Review
    #le = LabelEncoder()
    #Y = le.fit_transform(Y)
    #Y = Y.reshape(-1,1)
    return Y

def get_sequence_matrix_from_X(X, tok):
    tok.fit_on_texts(X)
    sequences = tok.texts_to_sequences(X)
    sequences_matrix = sequence.pad_sequences(sequences, maxlen=MAX_LEN)
    return sequences_matrix


sar_acc = process_df('movies_merged.csv')
X = get_X_from_df(sar_acc)
Y = get_Y_from_df(sar_acc)
TEST_SIZE = 0.2
RANDOM_STATE = 42
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [4]:
# Part 2 - Now let's make the ANN!


def ANN():
    # Initialising the ANN
    classifier = Sequential()

    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = (150)))
    # classifier.add(Dropout(rate = 0.1))

    # Adding the second hidden layer
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    # classifier.add(Dropout(rate = 0.1))

    # Adding the output layer
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

    # Compiling the ANN
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    # Fitting the ANN to the Training set
#     classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)
    return classifier

model = ANN()
model.sumary()
tok = Tokenizer(num_words=MAX_WORDS)
X_train_sequences = get_sequence_matrix_from_X(X_train, tok)
model.fit(X_train_sequences,
          Y_train,batch_size=10,
          epochs=100,
          validation_split=0.1,
          callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 44645 samples, validate on 4961 samples
Epoch 1/100
44645/44645 [==============================] - 5s 107us/step - loss: 0.2725 - acc: 0.9310 - val_loss: 0.2531 - val_acc: 0.9339
Epoch 2/100
44645/44645 [==============================] - 4s 94us/step - loss: 0.2548 - acc: 0.9311 - val_loss: 0.2472 - val_acc: 0.9339
Epoch 3/100
44645/44645 [==============================] - 5s 111us/step - loss: 0.2520 - acc: 0.9311 - val_loss: 0.2461 - val_acc: 0.9339
Epoch 4/100
44645/44645 [==============================] - 6s 123us/step - loss: 0.2511 - acc: 0.9311 - val_loss: 0.2446 - val_acc: 0.9339
Epoch 5/100
44645/44645 [==============================] - 6s 126us/step - loss: 0.2508 - acc: 0.9311 - val_loss: 0.2443 - val_acc: 0.9339
Epoch 6/100
44645/44645 [==============================] - 6s 141us/step - loss: 0.2507 - acc: 0.9311 - val_loss: 0.2436 - val_acc: 0.933